In [26]:
import numpy as np
import pyclesperanto_prototype as cle
from numba import njit
import raster_geometry as rg

In [27]:
def meshgrid_method():
    # create a 3D grid of points
    z, y, x = np.meshgrid(np.arange(-256, 256), np.arange(-256, 256), np.arange(-256, 256))

    # calculate the distance from each point to the center
    distance = np.sqrt(x**2 + y**2 + z**2)

    # create a numpy array with the same shape as distance
    sphere = np.zeros(distance.shape)

    # set points inside the sphere to 1
    sphere[distance < 256] = 1

    # display the result
    # print(np.unique(sphere))
    # print(sphere[50])
    # cle.imshow(sphere)

In [28]:
def fromfunction_method():
    def sphere_func(center, radius):
        def func(i, j, k):
            x, y, z = i - center[0], j - center[1], k - center[2]
            dist = np.sqrt(x**2 + y**2 + z**2)
            return dist <= radius
        return func

    # create a numpy array with the desired shape
    shape = (512, 512, 512)
    center = np.array(shape) // 2
    radius = 256

    # create the numpy array using np.fromfunction() and the sphere_func() function
    arr = np.fromfunction(sphere_func(center, radius), shape, dtype=np.int_)

    # display the result
    # print(np.unique(sphere))
    # print(sphere[50])
    # cle.imshow(sphere)
# fromfunction_method()

In [37]:
def numba_method():
    @njit()
    def sphere_func(center, radius, shape):
        arr = np.zeros(shape, dtype=np.uint8)
        for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    x, y, z = i - center[0], j - center[1], k - center[2]
                    dist = np.sqrt(x**2 + y**2 + z**2)
                    if dist <= radius:
                        arr[i,j,k] = 1
        return arr

    # create a numpy array with the desired shape
    shape = (400, 400, 400)
    center = np.array(shape) // 2
    radius = 200

    # create the numpy array using numba-accelerated sphere_func() function
    arr = sphere_func(center, radius, shape)
    # cle.imshow(arr)

numba_method()

In [34]:
def rastergeometry_method():
    radius = 50
    # Create an empty 3D NumPy array with dimensions equal to twice the radius plus one
    size = 2 * (radius + 1)

    # Calculate the midpoint of the sphere unit array
    midpoint = [size / 2] * 3

    # Generate a unit sphere using the rg library's superellipsoid function
    sphere = rg.nd_superellipsoid(size, radius, position=midpoint,
                                  rel_sizes=False, rel_position=False).astype(np.int_)
    
    # cle.imshow(sphere)
# rastergeometry_method()

In [35]:
## Tests
# print("meshgrid_method")
# %timeit meshgrid_method()
# print("fromfunction_method")
# %timeit fromfunction_method()
print("numba_method")
%time numba_method()
print("rastergeometry_method")
%time rastergeometry_method()


numba_method
CPU times: user 369 ms, sys: 16.3 ms, total: 386 ms
Wall time: 384 ms
rastergeometry_method
CPU times: user 1.79 ms, sys: 291 µs, total: 2.09 ms
Wall time: 2.09 ms
